In [1]:
from climakitae.core.data_interface import (
    get_data_options, 
    get_subsetting_options, 
    get_data
)
# import climakitae as ck

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.backends.backend_pdf import PdfPages
import time
from pyproj import Transformer
import geopandas as gpd
from shapely.geometry import Point
import contextily as cx
import climakitaegui as ckg

print('done')

done


# What are we solving for? 
## Temperature
### 1. Number of days in each OSHA catagory of high heat days at GWL 0.8, 1.5, 2.0
## Precipitation:
### 1. Number of days where daily precipitation exceeds 1,2,3,4 inches 



# Define the sets of variables
variables_units_list = [
    ("Precipitation (total)", "inches","Statistical","daily",[0.8,1.0,1.5,2.0]),
]

# Initialize dataset variables
ds = None

# Loop through each set of variables and process them
for variables_units in variables_units_list:
    variable, unit, downscale, timescale, GWL = variables_units
    print(variable, unit, downscale, timescale, GWL)
    print(f"Processing variable: {variable}")
    start_time = time.time()
    tmp_ds = get_data(
        variable=variable,
        units=unit,
        downscaling_method=downscale,
        resolution="3 km",
        timescale=timescale,
        cached_area="Southern California Edison",
        approach="Warming Level",
        warming_level_window=15,
        warming_level=GWL
    )
    print(f"data retreived in {time.time() - start_time:.2f} seconds.")
    if ds is None:
        ds = tmp_ds
    else:
        ds = xr.combine_by_coords([ds, tmp_ds])
    del tmp_ds

print(ds)


In [2]:
selections = ckg.Select()
selections.data_type = 'Gridded'
selections.area_subset = 'CA Electric Load Serving Entities (IOU & POU)'
selections.cached_area = ['Southern California Edison']
selections.timescale = 'hourly'
selections.variable_type = 'Derived Index'
selections.variable = 'NOAA Heat Index'
selections.resolution = '3 km'
selections.approach='Warming Level'
selections.warming_level = [0.8,1.5,2.0]

# selections.show()

heatidx = selections.retrieve()
print(heatidx)

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!! Returned data array is huge. Operations could take 10x to infinity longer than 1GB of data !!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

WARNING FOR WARMING LEVELS APPROACH
-----------------------------------
There may be NaNs in your data for certain simulation/warming level combinations if the warming level is not reached for that particular simulation before the year 2100. 

This does not mean you have missing data, but rather a feature of how the data is combined in retrieval to return a single data object. 

If you want to remove these empty simulations, it is recommended to first subset the data object by each individual warming level and then dropping NaN values.
<xarray.DataArray 'NOAA Heat Index' (warming_level: 3, time_delta: 262800,
                                     y: 240, x: 149, simulation: 8)> Size: 902GB
dask

In [3]:
# Calculate the hour of the year
hours_in_year = 365 * 24
hour_of_year = heatidx['time_delta'].values % hours_in_year
heatidx = heatidx.assign_coords(hour_of_year=('time_delta', hour_of_year))

# Calculate the month for each hour_of_year
hours_in_month = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]) * 24
cumulative_hours = np.cumsum(hours_in_month)

def hour_to_month(hour):
    return np.searchsorted(cumulative_hours, hour) + 1

months = hour_to_month(heatidx['hour_of_year'].values)
heatidx = heatidx.assign_coords(month=('time_delta', months))

# Calculate the day of the year for each hour_of_year
hours_in_day = 24

def hour_to_day_of_year(hour):
    return (hour // hours_in_day) + 1

day_of_year = hour_to_day_of_year(heatidx['hour_of_year'].values)
heatidx = heatidx.assign_coords(day_of_year=('time_delta', day_of_year))

# Calculate the total number of days in the dataset
total_days = np.arange(1, len(heatidx['time_delta']) // hours_in_day + 1)
total_days_repeated = np.repeat(total_days, hours_in_day)
heatidx = heatidx.assign_coords(total_days=('time_delta', total_days_repeated))

print(heatidx)
# time_delta = heatidx['time_delta'].values
# z = 0
# print(z)
# print('total days',total_days_repeated[z])
# print('day of year', day_of_year[z])
# print('month',months[z])
# print('hour of year', hour_of_year[z])
# print('time delta', time_delta[z])

<xarray.DataArray 'NOAA Heat Index' (warming_level: 3, time_delta: 262800,
                                     y: 240, x: 149, simulation: 8)> Size: 902GB
dask.array<concatenate, shape=(3, 262800, 240, 149, 8), dtype=float32, chunksize=(1, 1384, 54, 33, 1), chunktype=numpy.ndarray>
Coordinates: (12/15)
  * warming_level      (warming_level) float64 24B 0.8 1.5 2.0
  * x                  (x) float64 1kB -4.236e+06 -4.233e+06 ... -3.792e+06
  * y                  (y) float64 2kB 5.899e+05 5.929e+05 ... 1.307e+06
  * time_delta         (time_delta) float64 2MB -1.314e+05 ... 1.314e+05
    lakemask           (y, x) float32 143kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    landmask           (y, x) float32 143kB 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0
    ...                 ...
    centered_year      (warming_level, simulation) int64 192B 2005 2006 ... 2040
  * simulation         (simulation) <U44 1kB 'WRF_CESM2_r11i1p1f1_historical+...
    hour_of_year       (time_delta) float64 2MB 0.0 1.0 ... 

In [4]:
hi_threshold = 91
num_heatidx_hourly = (heatidx >= hi_threshold)
num_heatidx_hourly.name = 'Boolean if Heat Index is above threshold of {}°F'.format(hi_threshold)
print(num_heatidx_hourly)

<xarray.DataArray 'Boolean if Heat Index is above threshold of 91°F' (
                                                                      warming_level: 3,
                                                                      time_delta: 262800,
                                                                      y: 240,
                                                                      x: 149,
                                                                      simulation: 8)> Size: 226GB
dask.array<ge, shape=(3, 262800, 240, 149, 8), dtype=bool, chunksize=(1, 1384, 54, 33, 1), chunktype=numpy.ndarray>
Coordinates: (12/15)
  * warming_level      (warming_level) float64 24B 0.8 1.5 2.0
  * x                  (x) float64 1kB -4.236e+06 -4.233e+06 ... -3.792e+06
  * y                  (y) float64 2kB 5.899e+05 5.929e+05 ... 1.307e+06
  * time_delta         (time_delta) float64 2MB -1.314e+05 ... 1.314e+05
    lakemask           (y, x) float32 143kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [ ]:
heatidx_daily_number = num_heatidx_hourly.groupby('total_days').sum(dim='time_delta', skipna=True)
print(heatidx_daily_number)

In [ ]:
print('done')

In [56]:
# Calculate the day of the year based on the total days dimension
# days_in_year = 365
# day_of_year = (heatidx_daily_number['total_days'].values - 1) % days_in_year + 1
# heatidx_daily_number = heatidx_daily_number.assign_coords(day_of_year=('total_days', day_of_year))

# print(heatidx_daily_number)

heatidx_daily_climo = heatidx_daily_number.groupby('day_of_year').mean(dim='total_days', skipna=True)
print(heatidx_daily_climo)

<xarray.DataArray 'NOAA Heat Index' (warming_level: 4, day_of_year: 365,
                                     y: 240, x: 149, simulation: 8)> Size: 2GB
dask.array<transpose, shape=(4, 365, 240, 149, 8), dtype=float32, chunksize=(1, 1, 54, 33, 1), chunktype=numpy.ndarray>
Coordinates:
  * warming_level      (warming_level) float64 32B 0.8 1.0 1.5 2.0
  * x                  (x) float64 1kB -4.236e+06 -4.233e+06 ... -3.792e+06
  * y                  (y) float64 2kB 5.899e+05 5.929e+05 ... 1.307e+06
    lakemask           (y, x) float32 143kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    landmask           (y, x) float32 143kB 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0
    lat                (y, x) float32 143kB 31.49 31.5 31.52 ... 39.19 39.21
    lon                (y, x) float32 143kB -117.7 -117.7 ... -118.2 -118.2
    Lambert_Conformal  int64 8B 0
    centered_year      (warming_level, simulation) int64 256B 2005 2006 ... 2040
  * simulation         (simulation) <U44 1kB 'WRF_CESM2_r11i1p1f1_histo

In [58]:
print('done')
heatidx_daily_climo.name = 'number of hours {}°F'.format(hi_threshold)

done


In [ ]:
tmp = heatidx_daily_climo.sel(warming_level=1.5).load()
print(tmp)

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(18, 18))
lon2d, lat2d = np.meshgrid(heatidx_daily_climo.lon, heatidx_daily_climo.lat)
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
x_web, y_web = transformer.transform(lon2d, lat2d)
transformer_back = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
lon_back, lat_back = transformer_back.transform(x_web, y_web) 


temp_min = heatidx_daily_climo.min()
temp_max = heatidx_daily_climo.max()
ax = axs[0, 0]
mesh = ax.pcolormesh(x_web, y_web, tmp.sum(dim=day_of_year), cmap='YlOrRd',vmin=temp_min, vmax=temp_max)
cx.add_basemap(ax, crs="EPSG:3857", attribution_size=2)
ax.set_title(f'Number of hours at or over 91 degree F')
ax.set_ylabel('Latitude')
ax.set_xlabel('Longitude')
plt.colorbar(mesh, ax=ax, label=highest_5_percent_temperature_percentile_values[50].attrs['units'])
# Set the tick labels and reduce the number of ticks by half
ax.set_xticks(x_web[0, ::40])  # Adjust the step size to half
ax.set_xticklabels(np.round(lon_back[0, ::40], 2))
ax.set_yticks(y_web[::40, 0])  # Adjust the step size to half
ax.set_yticklabels(np.round(lat_back[::40, 0], 2))
# Rotate the tick labels
plt.xticks(rotation=45)
plt.yticks(rotation=45)
# Label the axes
plt.xlabel("Longitude")
plt.ylabel("Latitude")

In [ ]:
print('done')

In [ ]:
heatidx_daily_climo.to_netcdf('daily_NOAA_HI_GWL.nc')